In [1]:
from src.utils.utils import get_aug_img_transformer, validate_monte_carlo_prediction
import torch 
from sklearn.metrics import f1_score
from src.utils.utils import AverageMeter, coverage_size, get_monte_carlo_prediction_sets, pre_transform_dataset
import torchvision
from src.models.cifar10_conv_model import Cifar10ConvModel
import random


/Users/abdulrahman/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/abdulrahman/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: /Users/abdulrahman/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in: /Users/abdulrahman/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libc10.dylib
 in /Users/abdulrahman/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
test_set = torchvision.datasets.CIFAR10(train=False, root='../data/processed', transform=get_aug_img_transformer())
dataset = pre_transform_dataset(test_set)
test_loader =torch.utils.data.DataLoader(test_set, batch_size=len(test_set), shuffle=False)
indices = random.sample(range(len(dataset)), 9000)
val_dataset = torch.utils.data.Subset(indices=indices, dataset=dataset)
val_loader =torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False)

In [3]:
model = Cifar10ConvModel()
model.load_state_dict(torch.load("../models/trained_conv_cifar10_model.pt"))
model.eval()
model = torch.nn.DataParallel(model).cpu()

In [7]:
res = get_monte_carlo_prediction_sets(val_loader=val_loader, model=model, alpha=0.1, n_samples=len(indices), n_classes=10, forward_passes=15, randomized=True)

In [ ]:
out = validate_monte_carlo_prediction(val_loader=val_loader, predictions_sets=res[1])
out

(0.791681338028169, 2.609154929577465, 0.5466313819846647)

In [123]:
len(test_set)

10000